<a href="https://colab.research.google.com/github/Theivaprakasham/speechrecognition/blob/main/Speech2Text/Speech2Text_small_medium_Test_Inference_Librispeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FAIRSEQ S2T: Fast Speech-to-Text Model implementation**

The Speech2Text model was proposed in fairseq S2T: Fast Speech-to-Text Modeling with fairseq by Changhan Wang, Yun Tang, Xutai Ma, Anne Wu, Dmytro Okhonko, Juan Pino. It’s a transformer-based seq2seq (encoder-decoder) model designed for end-to-end Automatic Speech Recognition (ASR) and Speech Translation (ST). It uses a convolutional downsampler to reduce the length of speech inputs by 3/4th before they are fed into the encoder. The model is trained with standard autoregressive cross-entropy loss and generates the transcripts/translations autoregressively. Speech2Text has been fine-tuned on several datasets for ASR and ST: LibriSpeech, CoVoST 2, MuST-C.

**Experiments carried out by: Darshana S and Theivaprakasham H**

In [1]:
!wget https://raw.githubusercontent.com/Theivaprakasham/speechrecognition/main/Speech2Text/requirements.txt

--2021-11-21 14:33:13--  https://raw.githubusercontent.com/Theivaprakasham/speechrecognition/main/Speech2Text/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7656 (7.5K) [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]   7.48K  --.-KB/s    in 0s      

2021-11-21 14:33:13 (92.2 MB/s) - ‘requirements.txt’ saved [7656/7656]



In [3]:
!nvidia-smi

Sun Nov 21 14:35:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0-3
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

In [1]:
#Install required Dependencies
!pip install transformers"[speech, sentencepiece]"
!pip install datasets
!apt install git-lfs
!pip install jiwer

!pip install pandas sentencepiece
!pip install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install sox
!sudo apt-get install sox libsox-fmt-mp3

     |████████████████████████████████| 3.1 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 49.1 MB/s 
     |████████████████████████████████| 59 kB 6.9 MB/s 
     |████████████████████████████████| 3.3 MB 42.9 MB/s 
     |████████████████████████████████| 895 kB 49.4 MB/s 
     |████████████████████████████████| 1.2 MB 41.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 290 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 48.8 MB/s 
     |████████████████████████████████| 243 kB 60.1 MB/s 
     |████████████████████████████████| 132 kB 61.1 MB/s 
     |████████████████████████████████| 160 kB 55.7 MB/s 
     |████████████████████████████████| 271 kB 55.4 MB/s 
     |████████████████████████████████| 192 kB 50.0 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The f

# Loading Test Data from Librispeech clean Corpus

In [2]:
#Load Test clean data of LibriSpeech Dataset
from datasets import load_dataset, load_metric
librispeech_eval = load_dataset("librispeech_asr", "clean", split="test")  # change to "other" for other test dataset
wer = load_metric("wer")

Downloading:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/338M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset librispeech_asr downloaded and prepared to /root/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/f58a17d82b157feddbc407342a657fee7389b87c33d8331312edf7a604392896. Subsequent calls will reuse this data.


Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

# s2t-small-librispeech-asr model Inference on Test Data

In [4]:
#Load s2t-small-librispeech-asr

from transformers import Speech2TextForConditionalGeneration, Speech2TextProcessor
import soundfile as sf

model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr").to("cuda")
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr", do_upper_case=True)

def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

librispeech_eval = librispeech_eval.map(map_to_array)

def map_to_pred(batch):
    features = processor(batch["speech"], sampling_rate=16000, padding=True, return_tensors="pt")
    input_features = features.input_features.to("cuda")
    attention_mask = features.attention_mask.to("cuda")

    gen_tokens = model.generate(input_ids=input_features, attention_mask=attention_mask)
    batch["transcription"] = processor.batch_decode(gen_tokens, skip_special_tokens=True)
    return batch

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/407k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456 [00:00<?, ?B/s]

  0%|          | 0/2620 [00:00<?, ?ex/s]

In [5]:
%%time
#Testing s2t-small-librispeech-asr on Test Data on Librispeech
result = librispeech_eval.map(map_to_pred, batched=True, batch_size=8, remove_columns=["speech"])
print("Word Error Rate: ", wer.compute(predictions=result["transcription"], references=result["text"]))

  0%|          | 0/328 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/models/speech_to_text/modeling_speech_to_text.py:559: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  input_lengths = (input_lengths - 1) // 2 + 1


Word Error Rate:  0.04270009129640901
CPU times: user 8min 5s, sys: 5.27 s, total: 8min 11s
Wall time: 7min 56s


In [7]:
#Filtering 10 files from test Data
ten_data = librispeech_eval['file'][0:10]
data=[]
for i in ten_data:
  speech, _ = sf.read(i)
  data.append(speech)

In [8]:
%%time

##Time Taken for computing 10 audio images

features = processor(data, sampling_rate=16000, padding=True, return_tensors="pt")
input_features = features.input_features.to("cuda")
attention_mask = features.attention_mask.to("cuda")

gen_tokens = model.generate(input_ids=input_features, attention_mask=attention_mask)
result = processor.batch_decode(gen_tokens, skip_special_tokens=True)
print(result)

/usr/local/lib/python3.7/dist-packages/transformers/models/speech_to_text/modeling_speech_to_text.py:559: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  input_lengths = (input_lengths - 1) // 2 + 1


['HE HOPED THERE WOULD BE STEW FOR DINNER TURNIPS AND CARROTS AND BRUISED POTATOES AND FAT MUTTON PIECES TO BE LADLED OUT IN THICK PEPPERED FLOWER FAT AND SAUCE', 'STUFFED INTO YOU HIS BELLY COUNSELLED HIM', 'AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD LIGHT UP HERE AND THERE THE SQUALID QUARTER OF THE BROTHELS', 'HELLO BERTIE ANY GOOD IN YOUR MIND', 'NUMBER TEN FRESH NELLIE IS WAITING ON YOU GOOD NIGHT HUSBAND', "THE MUSIC CAME NEARER AND HE RECALLED THE WORDS THE WORDS OF SHELLEY'S FRAGMENT UPON THE MOON WANDERING COMPANIONLESS PALE FOR WEARINESS", 'THE DULL LIGHT FELL MORE FAINTLY UPON THE PAGE WHEREON ANOTHER EQUATION BEGAN TO UNFOLD ITSELF SLOWLY AND TO SPREAD ABROAD ITS WIDENING TAIL', 'A COLD LUCID INDIFFERENCE REIGNED IN HIS SOUL', 'THE CHAOS IN WHICH HIS ARDOR EXTINGUISHED ITSELF WAS A COLD INDIFFERENT KNOWLEDGE OF HIMSELF', 'AT MOST BY AN ALMS GIVEN TO A BEGGAR WHOSE BLESSING HE FLED FROM HE MIGHT HOPE WEARILY TO WIN FOR HIMSELF SOME MEASURE OF ACTUAL GRACE']
CPU times: user

# s2t-medium-librispeech-asr model Inference on Test Data

In [9]:
#loading pre-trained s2t-medium-librispeech-asr model

from transformers import Speech2TextForConditionalGeneration, Speech2TextProcessor
import soundfile as sf

model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-medium-librispeech-asr").to("cuda")
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-medium-librispeech-asr", do_upper_case=True)

def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

librispeech_eval = librispeech_eval.map(map_to_array)

def map_to_pred(batch):
    features = processor(batch["speech"], sampling_rate=16000, padding=True, return_tensors="pt")
    input_features = features.input_features.to("cuda")
    attention_mask = features.attention_mask.to("cuda")

    gen_tokens = model.generate(input_ids=input_features, attention_mask=attention_mask)
    batch["transcription"] = processor.batch_decode(gen_tokens, skip_special_tokens=True)
    return batch

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/272M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/407k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/458 [00:00<?, ?B/s]

  0%|          | 0/2620 [00:00<?, ?ex/s]

In [10]:
%%time
##Testing s2t-medium-librispeech-asr on Test Data on Librispeech
result = librispeech_eval.map(map_to_pred, batched=True, batch_size=8, remove_columns=["speech"])
print("Word Error Rate: ", wer.compute(predictions=result["transcription"], references=result["text"]))

  0%|          | 0/328 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/speech_to_text/modeling_speech_to_text.py:559: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  input_lengths = (input_lengths - 1) // 2 + 1


Word Error Rate:  0.03522519780888619
CPU times: user 9min 4s, sys: 4.42 s, total: 9min 8s
Wall time: 8min 52s


In [11]:
#Filtering 10 files from test Data
ten_data = librispeech_eval['file'][0:10]
data=[]
for i in ten_data:
  speech, _ = sf.read(i)
  data.append(speech)

In [12]:
%%time

##Time Taken for computing 10 audio images for s2t-medium-librispeech-asr inference

features = processor(data, sampling_rate=16000, padding=True, return_tensors="pt")
input_features = features.input_features.to("cuda")
attention_mask = features.attention_mask.to("cuda")

gen_tokens = model.generate(input_ids=input_features, attention_mask=attention_mask)
result = processor.batch_decode(gen_tokens, skip_special_tokens=True)
print(result)

/usr/local/lib/python3.7/dist-packages/transformers/models/speech_to_text/modeling_speech_to_text.py:559: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  input_lengths = (input_lengths - 1) // 2 + 1


['HE HOPED THERE WOULD BE STEW FOR DINNER TURNIPS AND CARROTS AND BRUISED POTATOES AND FAT MUTTON PIECES TO BE LADLED OUT IN THICK PEPPERED FLOUR FAT AND SAUCE', 'STUFFED INTO YOU HIS BELLY COUNSELLED HIM', 'AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD LIGHT UP HERE AND THERE THE SQUALID QUARTER OF THE BROTHELS', 'HELLO BERTIE ANY GOOD IN YOUR MIND', 'NUMBER TEN FRESH NELLIE IS WAITING ON YOU GOOD NIGHT HUSBAND', "THE MUSIC CAME NEARER AND HE RECALLED THE WORDS THE WORDS OF SHELLEY'S FRAGMENT UPON THE MOON WANDERING COMPANIONLESS PALE FOR WEARINESS", 'THE DULL LIGHT FELL MORE FAINTLY UPON THE PAGE WHEREON ANOTHER EQUATION BEGAN TO UNFOLD ITSELF SLOWLY AND TO SPREAD ABROAD ITS WIDENING TAIL', 'A COLD LUCID INDIFFERENCE REIGNED IN HIS SOUL', 'THE CHAOS IN WHICH HIS ARDOR EXTINGUISHED ITSELF WAS A COLD INDIFFERENT KNOWLEDGE OF HIMSELF', 'AT MOST BY AN ALMS GIVEN TO A BEGGAR WHOSE BLESSING HE FLED FROM HE MIGHT HOPE WEARILY TO WIN FOR HIMSELF SOME MEASURE OF ACTUAL GRACE']
CPU times: user 

# Requirements

In [13]:
!pip freeze

absl-py==0.12.0
aiohttp==3.8.1
aiosignal==1.2.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.1.0
appdirs==1.4.4
argcomplete==1.12.3
argon2-cffi==21.1.0
arviz==0.11.4
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
async-timeout==4.0.1
asynctest==0.13.0
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.2.0
audioread==2.1.9
autograd==1.3
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==4.1.0
blis==0.4.1
bokeh==2.3.3
Bottleneck==1.3.2
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.10
cached-property==1.5.2
cachetools==4.2.4
catalogue==1.0.0
certifi==2021.10.8
cffi==1.15.0
cftime==1.5.1.1
chardet==3.0.4
charset-normalizer==2.0.7
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==2.0.6
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.3.2
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cupy-cuda111==9.4.0
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.24
daft==0.0.4
dask==2.12.0
datascience==0.10.6
datasets

In [17]:
!pip install absl-py==0.12.0 aiohttp==3.8.1 aiosignal==1.2.0 alabaster==0.7.12 albumentations==0.1.12 altair==4.1.0 appdirs==1.4.4 argcomplete==1.12.3 argon2-cffi==21.1.0 arviz==0.11.4 astor==0.8.1 astropy==4.3.1 astunparse==1.6.3 async-timeout==4.0.1 asynctest==0.13.0 atari-py==0.2.9 atomicwrites==1.4.0 attrs==21.2.0 audioread==2.1.9 autograd==1.3 Babel==2.9.1 backcall==0.2.0 beautifulsoup4==4.6.3 bleach==4.1.0 blis==0.4.1 bokeh==2.3.3 Bottleneck==1.3.2 branca==0.4.2 bs4==0.0.1 CacheControl==0.12.10 cached-property==1.5.2 cachetools==4.2.4 catalogue==1.0.0 certifi==2021.10.8 cffi==1.15.0 cftime==1.5.1.1 chardet==3.0.4 charset-normalizer==2.0.7 click==7.1.2 cloudpickle==1.3.0 cmake==3.12.0 cmdstanpy==0.9.5 colorcet==2.0.6 colorlover==0.3.0 community==1.0.0b1 contextlib2==0.5.5 convertdate==2.3.2 coverage==3.7.1 coveralls==0.5 crcmod==1.7 cufflinks==0.17.3 cupy-cuda111==9.4.0 cvxopt==1.2.7 cvxpy==1.0.31 cycler==0.11.0 cymem==2.0.6 Cython==0.29.24 daft==0.0.4 dask==2.12.0 datascience==0.10.6 datasets==1.15.1 debugpy==1.0.0 decorator==4.4.2 defusedxml==0.7.1 descartes==1.1.0 dill==0.3.4 distributed==1.25.3 dlib @ file:///dlib-19.18.0-cp37-cp37m-linux_x86_64.whl dm-tree==0.1.6 docopt==0.6.2 docutils==0.17.1 dopamine-rl==1.0.5 earthengine-api==0.1.290 easydict==1.9 ecos==2.0.7.post1 editdistance==0.5.3 en-core-web-sm @ https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.5/en_core_web_sm-2.2.5.tar.gz entrypoints==0.3 ephem==4.1 et-xmlfile==1.1.0 fa2==0.3.5 fastai==1.0.61 fastdtw==0.3.4 fastprogress==1.0.0 fastrlock==0.8 fbprophet==0.7.1 feather-format==0.4.1 filelock==3.4.0 firebase-admin==4.4.0 fix-yahoo-finance==0.0.22 Flask==1.1.4 flatbuffers==2.0 folium==0.8.3 fonttools==4.28.1 frozenlist==1.2.0 fsspec==2021.11.0 future==0.16.0 gast==0.4.0 GDAL==2.2.2 gdown==3.6.4 gensim==3.6.0 geographiclib==1.52 geopy==1.17.0 gin-config==0.5.0 glob2==0.7 google==2.0.3 google-api-core==1.26.3 google-api-python-client==1.12.8 google-auth==1.35.0 google-auth-httplib2==0.0.4 google-auth-oauthlib==0.4.6 google-cloud-bigquery==1.21.0 google-cloud-bigquery-storage==1.1.0 google-cloud-core==1.0.3 google-cloud-datastore==1.8.0 google-cloud-firestore==1.7.0 google-cloud-language==1.2.0 google-cloud-storage==1.18.1 google-cloud-translate==1.5.0 google-colab @ file:///colabtools/dist/google-colab-1.0.0.tar.gz google-pasta==0.2.0 google-resumable-media==0.4.1 googleapis-common-protos==1.53.0 googledrivedownloader==0.4 graphviz==0.10.1 greenlet==1.1.2 grpcio==1.42.0 gspread==3.0.1 gspread-dataframe==3.0.8 gym==0.17.3 h5py==3.1.0 HeapDict==1.0.1 hijri-converter==2.2.2 holidays==0.10.5.2 holoviews==1.14.6 html5lib==1.0.1 httpimport==0.5.18 httplib2==0.17.4 httplib2shim==0.0.3 huggingface-hub==0.1.2 humanize==0.5.1 hyperopt==0.1.2 ideep4py==2.0.0.post3 idna==2.10 imageio==2.4.1 imagesize==1.3.0 imbalanced-learn==0.8.1 imblearn==0.0 imgaug==0.2.9 importlib-metadata==4.8.2 importlib-resources==5.4.0 imutils==0.5.4 inflect==2.1.0 iniconfig==1.1.1 intel-openmp==2021.4.0 intervaltree==2.1.0 ipykernel==4.10.1 ipython==5.5.0 ipython-genutils==0.2.0 ipython-sql==0.3.9 ipywidgets==7.6.5 itsdangerous==1.1.0 jax==0.2.21 jaxlib @ https://storage.googleapis.com/jax-releases/cuda111/jaxlib-0.1.71+cuda111-cp37-none-manylinux2010_x86_64.whl jdcal==1.4.1 jedi==0.18.1 jieba==0.42.1 Jinja2==2.11.3 jiwer==2.3.0 joblib==1.1.0 jpeg4py==0.1.4 jsonschema==2.6.0 jupyter==1.0.0 jupyter-client==5.3.5 jupyter-console==5.2.0 jupyter-core==4.9.1 jupyterlab-pygments==0.1.2 jupyterlab-widgets==1.0.2 kaggle==1.5.12 kapre==0.3.6 keras==2.7.0 Keras-Preprocessing==1.1.2 keras-vis==0.4.1 kiwisolver==1.3.2 korean-lunar-calendar==0.2.1 libclang==12.0.0 librosa==0.8.1 lightgbm==2.2.3 llvmlite==0.34.0 lmdb==0.99 LunarCalendar==0.0.9 lxml==4.2.6 Markdown==3.3.6 MarkupSafe==2.0.1 matplotlib==3.2.2 matplotlib-inline==0.1.3 matplotlib-venn==0.11.6 missingno==0.5.0 mistune==0.8.4 mizani==0.6.0 mkl==2019.0 mlxtend==0.14.0 more-itertools==8.11.0 moviepy==0.2.3.5 mpmath==1.2.1 msgpack==1.0.2 multidict==5.2.0 multiprocess==0.70.12.2 multitasking==0.0.10 murmurhash==1.0.6 music21==5.5.0 natsort==5.5.0 nbclient==0.5.8 nbconvert==5.6.1 nbformat==5.1.3 nest-asyncio==1.5.1 netCDF4==1.5.8 networkx==2.6.3 nibabel==3.0.2 nltk==3.2.5 notebook==5.3.1 numba==0.51.2 numexpr==2.7.3 numpy==1.19.5 nvidia-ml-py3==7.352.0 oauth2client==4.1.3 oauthlib==3.1.1 okgrade==0.4.3 opencv-contrib-python==4.1.2.30 opencv-python==4.1.2.30 openpyxl==2.5.9 opt-einsum==3.3.0 osqp==0.6.2.post0 packaging==21.3 palettable==3.3.0 pandas==1.1.5 pandas-datareader==0.9.0 pandas-gbq==0.13.3 pandas-profiling==1.4.1 pandocfilters==1.5.0 panel==0.12.1 param==1.12.0 parso==0.8.2 pathlib==1.0.1 patsy==0.5.2 pep517==0.12.0 pexpect==4.8.0 pickleshare==0.7.5 Pillow==7.1.2 pip-tools==6.2.0 plac==1.1.3 plotly==4.4.1 plotnine==0.6.0 pluggy==0.7.1 pooch==1.5.2 portpicker==1.3.9 prefetch-generator==1.0.1 preshed==3.0.6 prettytable==2.4.0 progressbar2==3.38.0 prometheus-client==0.12.0 promise==2.3 prompt-toolkit==1.0.18 protobuf==3.17.3 psutil==5.4.8 psycopg2==2.7.6.1 ptyprocess==0.7.0 py==1.11.0 pyarrow==3.0.0 pyasn1==0.4.8 pyasn1-modules==0.2.8 pycocotools==2.0.2 pycparser==2.21 pyct==0.4.8 pydata-google-auth==1.2.0 pydot==1.3.0 pydot-ng==2.0.0 pydotplus==2.0.2 PyDrive==1.3.1 pyemd==0.5.1 pyerfa==2.0.0.1 pyglet==1.5.0 Pygments==2.6.1 pygobject==3.26.1 pymc3==3.11.4 PyMeeus==0.5.11 pymongo==3.12.1 pymystem3==0.2.0 PyOpenGL==3.1.5 pyparsing==3.0.6 pyrsistent==0.18.0 pysndfile==1.3.8 PySocks==1.7.1 pystan==2.19.1.1 pytest==3.6.4 python-apt==0.0.0 python-chess==0.23.11 python-dateutil==2.8.2 python-Levenshtein==0.12.2 python-louvain==0.15 python-slugify==5.0.2 python-utils==2.5.6 pytz==2018.9 pyviz-comms==2.1.0 PyWavelets==1.2.0 PyYAML==6.0 pyzmq==22.3.0 qdldl==0.1.5.post0 qtconsole==5.2.0 QtPy==1.11.2 regex==2019.12.20 requests==2.23.0 requests-oauthlib==1.3.0 resampy==0.2.2 retrying==1.3.3 rpy2==3.4.5 rsa==4.7.2 sacremoses==0.0.46 scikit-image==0.18.3 scikit-learn==1.0.1 scipy==1.4.1 screen-resolution-extra==0.0.0 scs==2.1.4 seaborn==0.11.2 semver==2.13.0 Send2Trash==1.8.0 sentencepiece==0.1.96 setuptools-git==1.2 setuptools-scm==6.3.2 Shapely==1.8.0 simplegeneric==0.8.1 six==1.15.0 sklearn==0.0 sklearn-pandas==1.8.0 smart-open==5.2.1 snowballstemmer==2.2.0 sortedcontainers==2.4.0 SoundFile==0.10.3.post1 spacy==2.2.4 Sphinx==1.8.6 sphinxcontrib-serializinghtml==1.1.5 sphinxcontrib-websupport==1.2.4 SQLAlchemy==1.4.27 sqlparse==0.4.2 srsly==1.0.5 statsmodels==0.10.2 sympy==1.7.1 tables==3.4.4 tabulate==0.8.9 tblib==1.7.0 tensorboard==2.7.0 tensorboard-data-server==0.6.1 tensorboard-plugin-wit==1.8.0 tensorflow @ file:///tensorflow-2.7.0-cp37-cp37m-linux_x86_64.whl tensorflow-datasets==4.0.1 tensorflow-estimator==2.7.0 tensorflow-gcs-config==2.7.0 tensorflow-hub==0.12.0 tensorflow-io-gcs-filesystem==0.22.0 tensorflow-metadata==1.4.0 tensorflow-probability==0.15.0 termcolor==1.1.0 terminado==0.12.1 testpath==0.5.0 text-unidecode==1.3 textblob==0.15.3 Theano-PyMC==1.1.2 thinc==7.4.0 threadpoolctl==3.0.0 tifffile==2021.11.2 tokenizers==0.10.3 toml==0.10.2 tomli==1.2.2 toolz==0.11.2 torch==1.10.0+cu113 torchaudio==0.10.0+cu113 torchsummary==1.5.1 torchtext==0.11.0 torchvision==0.11.1+cu113 tornado==5.1.1 tqdm==4.62.3 traitlets==5.1.1 transformers==4.12.5 tweepy==3.10.0 typeguard==2.7.1 typing-extensions==3.10.0.2 tzlocal==1.5.1 uritemplate==3.0.1 urllib3==1.24.3 vega-datasets==0.9.0 wasabi==0.8.2 wcwidth==0.2.5 webencodings==0.5.1 Werkzeug==1.0.1 widgetsnbextension==3.5.2 wordcloud==1.5.0 wrapt==1.13.3 xarray==0.18.2 xgboost==0.90 xkit==0.0.0 xlrd==1.1.0 xlwt==1.3.0 xxhash==2.0.2 yarl==1.7.2 yellowbrick==1.3.post1 zict==2.0.0 zipp==3.6.0

ERROR: Invalid requirement: '@'


In [20]:
pip freeze > requirements.txt

# Reference


1.   C. Wang, Y. Tang, X. Ma, A. Wu, D. Okhonko, and J. Pino, “Fairseq {S}2{T}: Fast Speech-to-Text Modeling with Fairseq,” in Proceedings of the 1st Conference of the Asia-Pacific Chapter of the Association for Computational Linguistics and the 10th International Joint Conference on Natural Language Processing: System Demonstrations, Dec. 2020, pp. 33–39, [Online]. Available: https://aclanthology.org/2020.aacl-demo.6.
[arxiv](https://arxiv.org/abs/2010.05171)

2. **Code:** Speech2Text Hugging Face implementation  [Speech2Text Hugging Face Implementation](https://huggingface.co/transformers/model_doc/speech_to_text.html)
